In [1]:
#Must define port that the Arduino is plugged into such as 'COM4'

port='COM4'

In [2]:
#Sensor_values_capture captures readings from steering sensor and throttle onto Arduino board

import sys
from pyfirmata import Arduino, util
import time

In [5]:
# Steering sensor Pin #1,  The min and max can be edited after a calibration step

mapValues = False  

steerSensorPin = 1
#minSteerSensorValue = 0.1253
#maxSteerSensorValue = 0.3795
minSteerSensorValue = 0.119
maxSteerSensorValue = 0.4378

In [6]:
# Throttle Sensor Pin #2, the min and max can be edited after a calibration step

throttleSensorPin = 2
minThrottleSensorValue = 0.1700
maxThrottleSensorValue = 0.7720

minThrottlePWM = 0 
maxThrottlePWM = 1.5 #real max 1

minSteeringPWM = 0 
maxSteeringPWM = 0.6 #real max 1

reverseSteering = -1 # make 1 to reverse steering

In [7]:
#Variables for some items.  I changed the arduinoNano variable from the original build to arduinoUno variable because I happen
# to have an UNO.  The Nano should be big enough to handle this build but if you add more sensors, other inputs like brakes, 
#or robot arms, may need Uno.  Variable name does not matter if you use a Nano or Uno.


arduinoUno = None
leftPWM = None
rightPWM = None 
rightEnable = 3
leftEnable = 5 
throttlePin = None

#This code block appears to confirm if Arduino is connected and communicating as well as sensors.

def connect(port):
    global arduinoUno, leftPWM, rightPWM, leftEnable, rightEnable, throttlePin
    if arduinoUno == None:
        arduinoUno = Arduino(port)
        it = util.Iterator(arduinoUno)
        it.start()
        arduinoUno.analog[steerSensorPin].enable_reporting()
        arduinoUno.analog[throttleSensorPin].enable_reporting()
        leftPWM = arduinoUno.get_pin('d:6:p')
        rightPWM = arduinoUno.get_pin('d:9:p')
        throttlePin = arduinoUno.get_pin('d:10:p')
        arduinoUno.digital[rightEnable].write(1)
        arduinoUno.digital[leftEnable].write(1)
        return "success"
    else:
        return "error"

In [8]:
#This code disconnects the arduino

def disconnect():
    global arduinoUno
    board.digital[rightEnable].write(0)
    board.digital[leftEnable].write(0)
    writeValue(0,0)
    arduinoUno.exit()

In [9]:
#This code actually reads the value from the throttle and sensor.  It returns the steering position and throttle position


def readValue():
    global arduinoUno
    if arduinoUno != None:
        steerPosition = arduinoUno.analog[steerSensorPin].read()  
        throttlePosition = arduinoUno.analog[throttleSensorPin].read() 
        #TODO translate value from min max to -1 1 and 0 1 for throttle
        if(steerPosition != None and throttlePosition != None):
            if(mapValues):
                mappedSteeringPosition = mapSteerPisition(steerPosition, minSteerSensorValue, maxSteerSensorValue, -1, 1)
                mappedThrottlePosition = mapThrottle(throttlePosition, minThrottleSensorValue, maxThrottleSensorValue, 0, 1)
                #print("readed steeringposition > " + str(mappedSteeringPosition) + " readed throttle " + str(mappedThrottlePosition))
                return((mappedSteeringPosition * reverseSteering,mappedThrottlePosition))
            else:
                #print("readed steeringposition > " + str(steerPosition) + " readed throttle " + str(throttlePosition))
                return((steerPosition,throttlePosition))
    else:
        return("error")


In [10]:
#This writes values for steering and throttle based on sensor readings

def writeValue(left,right,throttle):
    global leftPWM, rightPWM, leftEnable, rightEnable, throttlePin

    mappedLeft = 0
    mappedRight = 0

    if left > 0:
        rightPWM.write(0)
        mappedLeft = round(map(left, -1, 1, minSteeringPWM, maxSteeringPWM),3)
        leftPWM.write(mappedLeft)
    elif right > 0:
        leftPWM.write(0)
        mappedRight = round(map(right, -1, 1, minSteeringPWM, maxSteeringPWM),3)
        rightPWM.write(mappedRight)
    else:
        print("left right commando incorreect")

    mappedThrottle = round(map(throttle, 0, 1, minThrottlePWM, maxThrottlePWM),3)
    #print("writing values left> " + str(mappedLeft) + " right> " + str(mappedRight) + " throttle> " + str(mappedThrottle))
    throttlePin.write(mappedThrottle)
    

In [11]:
#This returns the value for the throttle



def mapThrottle (value, fromSource,  toSource,  fromTarget,  toTarget):
    mappedThottle = (value - fromSource) / (toSource - fromSource) * (toTarget - fromTarget) + fromTarget
    if mappedThottle > 1:
        mappedThottle = 1
    elif mappedThottle < 0:
        mappedThottle = 0
    return mappedThottle

In [9]:
#This returns the mapped steering position

def mapSteerPisition (value, fromSource,  toSource,  fromTarget,  toTarget):
    mappedSteerPisition = (value - fromSource) / (toSource - fromSource) * (toTarget - fromTarget) + fromTarget
    if mappedSteerPisition > 1:
        mappedSteerPisition = 1
    elif mappedSteerPisition < -1:
        mappedSteerPisition = -1
    return mappedSteerPisition

def map(value, fromSource,  toSource,  fromTarget,  toTarget):
    return (value - fromSource) / (toSource - fromSource) * (toTarget - fromTarget) + fromTarget
    

#connect('/dev/ttyUSB0')
#while(True):
    #writeValue(1,0)
    #time.sleep(10)
    #disconnect()